In [1]:
from ophelia.start import Ophelia

In [2]:
ophelia = Ophelia("Ophelia-Test", True)

sc = ophelia.Spark.build_spark_context()
spark = ophelia.SparkSession

02:15:00.586 Ophelia [TAPE] +---------------------------------------------------------------------+
02:15:00.586 Ophelia [INFO] | Hello! This package is for data mining & ml pipelines using PySpark |
02:15:00.586 Ophelia [INFO] | Welcome to Ophelia Spark miner engine                               |
02:15:00.586 Ophelia [INFO] | Package Version Ophelia.0.0.1                                       |
02:15:00.586 Ophelia [WARN] | V for Vendata...                                                    |
02:15:00.586 Ophelia [TAPE] +---------------------------------------------------------------------+
02:15:00.586 Ophelia [WARN]                   - Ophelia a pyspark miner -                      
02:15:00.586 Ophelia [MASK]   █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █    
02:15:00.587 Ophelia [MASK]   █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █ █ █    
02:15:00.587 Ophelia [MASK]   █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █    
02:15:00.587 Oph

In [3]:
import pandas as pd

dataset = {
    'Product': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Year': [2010, 2010, 2010, 2011, 2011, 2011, 2012, 2012, 2012],
    'Revenue': [100, 200, 300, 110, 190, 320, 120, 220, 350]
}

df = spark.createDataFrame(pd.DataFrame(data=dataset))
df.show(5)

+-------+----+-------+
|Product|Year|Revenue|
+-------+----+-------+
|      A|2010|    100|
|      B|2010|    200|
|      C|2010|    300|
|      A|2011|    110|
|      B|2011|    190|
+-------+----+-------+
only showing top 5 rows



In [4]:
from ophelia.functions import Shape

df.Shape

(9, 3)

In [5]:
from ophelia.functions import PctChange

df.pctChange().show(5, False)

TypeError: 'JavaMember' object is not iterable

In [ ]:
df.pctChange(periods=2).na.fill(0).show(5, False)

In [ ]:
df.pctChange(partition_by="Product", order_by="Year", pct_cols="Revenue").na.fill(0).show(5, False)

In [ ]:
df.pctChange(partition_by="Product", order_by="Year", pct_cols=["Year", "Revenue"]).na.fill(0).show(5, False)

In [ ]:
from ophelia.functions import Reshape

# We can choose either one column to aggregate with one single operation
agg_dict_test = {'Revenue': 'max'}

# Or moreTthan one column and more than one operation
_agg_dict_test = {
    'Revenue': 'mean, count, sum, stddev, var, min',
    'Year': 'count, sum',
    'Product': 'count'
}

df.toWide(group_by="Product", pivot_col="Year", agg_dict=agg_dict_test).orderBy("Product_Year").show(5, False)

In [ ]:
bank_dataset = {
    'age': [59, 56, 41, 55, 54, 42, 56, 60, 37, 28],
    'job': ['admin.', 'admin.', 'technician', 'services', 'admin.', 'management', 'management', 'retired', 'technician', 'services'],
    'marital': ['married', 'married', 'married', 'married', 'married', 'single', 'married', 'divorced', 'married', 'single'],
    'education': ['secondary', 'secondary', 'secondary', 'secondary', 'tertiary', 'tertiary', 'tertiary', 'secondary', 'secondary', 'secondary'],
    'default': ['no', 'yes', 'no', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes'],
    'balance': [2343, 45, 1270, 2476, 184, 0, 830, 545, 1, 5090],
    'housing': ['yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes'],
    'loan': ['yes', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no'],
    'day': [5, 5, 5, 5, 5, 5, 6, 6, 6, 6],
    'month': ['may', 'may', 'jun', 'jul', 'jan', 'dec', 'oct', 'sep', 'jan', 'jul'],
    'duration': [1042, 1467, 1389, 579, 673, 562, 1201, 1030, 608, 1297],
    'campaign': [1, 1, 1, 1, 2, 2, 1, 1, 1, 3]
}

bank_df = spark.createDataFrame(pd.DataFrame(bank_dataset))
bank_df.show()

In [ ]:
from ophelia.functions import Selects
from ophelia.generic import regex_expr

bank_df.selectRegex(regex_expr(['day', 'ous', 'aig'])).show(5, False)

In [ ]:
bank_df.selectContains(['day', 'ous', 'aig']).show(5, False)

In [ ]:
from ophelia.generic import string_match

bank_df.where(string_match('housing == yes')).crosstab('education', 'campaign').orderBy('education_campaign').show()

In [6]:
test_agg_dict_ = {
    'balance': 'sum'
}
bank_df.where(string_match('housing == yes')).toWide('education', 'campaign', test_agg_dict_).orderBy('education_campaign').show()

NameError: name 'bank_df' is not defined

In [19]:
from ophelia.functions import MapItem

map_values = {
    'A': '0',
    'B': '1',
    'C': '2'
}

df.mapItem('Product', 'bin_product', map_values).show()

+-------+----+-------+-----------+
|Product|Year|Revenue|bin_product|
+-------+----+-------+-----------+
|      A|2010|    100|          0|
|      B|2010|    200|          1|
|      C|2010|    300|          2|
|      A|2011|    110|          0|
|      B|2011|    190|          1|
|      C|2011|    320|          2|
|      A|2012|    120|          0|
|      B|2012|    220|          1|
|      C|2012|    350|          2|
+-------+----+-------+-----------+

